# Segmenting and Clustering Neighborhoods in Toronto

### Import the libraries.

In [1]:
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Send the GET request for the website.

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_html_doc = requests.get(url)

html_doc=raw_html_doc.text

### Use BeautifulSoup to parse the data and get the table of neighborhood postal codes.

In [3]:
tree = BeautifulSoup(html_doc,"lxml")
table_tag = tree.select("table")[0]
tab_data = [[item.text for item in row_data.select("th,td")]
                for row_data in table_tag.select("tr")]

### Create the dataframe.

In [4]:
# Create the dataframe

# Define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood'] 

# Convert tab_data  to dataframe.
neighborhoods_data = pd.DataFrame(tab_data,columns=column_names)
neighborhoods_data.head()

,PostalCode,Borough,Neighborhood
0,Postcode,Borough,Neighbourhood\n
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


### Get the Geospatial data.

In [5]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')

df_geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge the Geospatial data into the neighborhoods data.

In [6]:
neighborhoods = pd.merge(neighborhoods_data, df_geo, on='PostalCode', how='left')

# Remove rows that have NA values.
neighborhoods = neighborhoods.dropna(axis=0)

# Remove \n from data.
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].replace('\n',' ', regex=True) 
#neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].str.replace(r'\n', '')

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)
neighborhoods.head()

The dataframe has 11 boroughs and 211 neighborhoods.


,PostalCode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.753259,-79.329656
4,M4A,North York,Victoria Village,43.725882,-79.315572
5,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
6,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
7,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [7]:
 neighborhoods.groupby('Borough').count()

,PostalCode,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,17,17,17,17
Downtown Toronto,37,37,37,37
East Toronto,7,7,7,7
East York,6,6,6,6
Etobicoke,45,45,45,45
Mississauga,1,1,1,1
North York,38,38,38,38
Queen's Park,1,1,1,1
Scarborough,37,37,37,37


### Get the borough of Central Toronto.

In [9]:
central_toronto_data = neighborhoods[neighborhoods['Borough'] == 'Central Toronto'].reset_index(drop=True)
central_toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,Forest Hill North,43.696948,-79.411307
4,M5P,Central Toronto,Forest Hill West,43.696948,-79.411307


### Get the geographical coordinates of Toronto.

In [10]:
address = 'Toronto'
geolocator = Nominatim(user_agent="cd_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create map of Central Toronto using latitude and longitude values.

In [11]:
# create map of central toronto using latitude and longitude values
map_central_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(central_toronto_data['Latitude'], central_toronto_data['Longitude'], central_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central_toronto)  
    
map_central_toronto

### Define Foursquare Credentials and Version

### Explore the first neighborhood in our dataframe.

In [104]:
central_toronto_data.loc[0, 'Neighborhood']

# Get the neighborhood's latitude and longitude values.
neighborhood_latitude = central_toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = central_toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

### Get the neighborhood's name.
neighborhood_name = central_toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park  are 43.7280205, -79.3887901.


### Get the top 100 venues that are within a radius of 500 meters.

In [105]:
# Create the GET request URL.

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XAJLSH3R3GW5PNAARSDELOM4E4OIZTV25NJAEZ4RTSQADLBT&client_secret=QCM5NXJAO1RWDDMVMSNIM0E5AEHNDMJJRWTFWLAK022YKKJU&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [106]:
results = requests.get(url).json()

### Clean the json and structure it into a pandas dataframe.

In [107]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# clean the json and structure it into a pandas dataframe.

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


### Explore neighborhoods in Central Toronto.

In [108]:
# Create a function to repeat the same process to all the neighborhoods in Central Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get the venues in the neighborhoods.

In [109]:
central_toronto_venues = getNearbyVenues(names=central_toronto_data['Neighborhood'],
                                   latitudes=central_toronto_data['Latitude'],
                                   longitudes=central_toronto_data['Longitude']
                                  )

print(central_toronto_venues.shape)

Lawrence Park 
Roselawn 
Davisville North 
Forest Hill North 
Forest Hill West 
North Toronto West 
The Annex 
North Midtown 
Yorkville 
Davisville 
Moore Park 
Summerhill East 
Deer Park 
Forest Hill SE 
Rathnelly 
South Hill 
Summerhill West 
(237, 7)


In [110]:
central_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
4,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park


In [111]:
central_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,37,37,37,37,37,37
Davisville North,11,11,11,11,11,11
Deer Park,15,15,15,15,15,15
Forest Hill North,4,4,4,4,4,4
Forest Hill SE,15,15,15,15,15,15
Forest Hill West,4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
Moore Park,4,4,4,4,4,4
North Midtown,25,25,25,25,25,25


In [112]:
# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(central_toronto_venues['Venue Category'].unique())))

There are 63 uniques categories.


### Analyze each neighborhood.

In [113]:
# Analyze Each Neighborhood
# one hot encoding
central_toronto_onehot = pd.get_dummies(central_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
central_toronto_onehot['Neighborhood'] = central_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [central_toronto_onehot.columns[-1]] + list(central_toronto_onehot.columns[:-1])
central_toronto_onehot = central_toronto_onehot[fixed_columns]

central_toronto_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dance Studio,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Medical Center,Metro Station,Mexican Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [114]:
central_toronto_onehot.shape

(237, 64)

In [115]:
central_toronto_grouped = central_toronto_onehot.groupby('Neighborhood').mean().reset_index()
central_toronto_grouped

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dance Studio,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Medical Center,Metro Station,Mexican Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.027027,0.00,0.000000,0.000000,0.027027,0.027027,0.000000,0.054054,0.00,0.000000,0.000000,0.054054,0.000000,0.00,0.027027,0.081081,0.027027,0.027027,0.000000,0.027027,0.000000,0.027027,0.0,0.000000,0.027027,0.027027,0.027027,0.00,0.000000,0.027027,0.054054,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.027027,0.000000,0.027027,0.081081,0.00,0.000000,0.000000,0.054054,0.000000,0.081081,0.027027,0.000000,0.000000,0.000000,0.000000,0.054054,0.000000,0.027027,0.027027,0.027027,0.00,0.00,0.000000,0.000000
1,Davisville North,0.000000,0.090909,0.00,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,0.00,0.000000,0.090909,0.000000,0.000000,0.00,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.0,0.000000,0.000000,0.000000,0.090909,0.00,0.090909,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.090909,0.00,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000
2,Deer Park,0.066667,0.000000,0.00,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.133333,0.066667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.066667,0.00,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.00,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.066667,0.000000
3,Forest Hill North,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.000000
4,Forest Hill SE,0.066667,0.000000,0.00,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.133333,0.066667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.066667,0.00,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.00,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.066667,0.000000
5,Forest Hill West,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [116]:
central_toronto_grouped.shape

(17, 64)

### Write a function to sort the venues in descending order.

In [117]:
# write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create the new dataframe and display the top 10 venues for each neighborhood.

In [118]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [119]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = central_toronto_grouped['Neighborhood']

for ind in np.arange(central_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(central_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Restaurant,Café,Coffee Shop,Sushi Restaurant,Park,Dance Studio
1,Davisville North,Sandwich Place,Asian Restaurant,Gym,Breakfast Spot,Dance Studio,Burger Joint,Hotel,Park,Pizza Place,Food & Drink Shop
2,Deer Park,Pub,Coffee Shop,American Restaurant,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Medical Center,Pizza Place,Convenience Store
3,Forest Hill North,Sushi Restaurant,Trail,Jewelry Store,Park,Food & Drink Shop,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Yoga Studio
4,Forest Hill SE,Pub,Coffee Shop,American Restaurant,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Medical Center,Pizza Place,Convenience Store


### Perform cluster analysis using k-means.

In [120]:
# Cluster Neighborhoods
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

central_toronto_grouped_clustering = central_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(central_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 3, 0, 3, 4, 2, 0, 0], dtype=int32)

In [121]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

central_toronto_merged = central_toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
central_toronto_merged = central_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

central_toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Bus Line,Park,Swim School,Yoga Studio,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Garden,Yoga Studio,Dance Studio,History Museum,Gym,Greek Restaurant,Gourmet Shop,Gift Shop,Fried Chicken Joint,Food & Drink Shop
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Sandwich Place,Asian Restaurant,Gym,Breakfast Spot,Dance Studio,Burger Joint,Hotel,Park,Pizza Place,Food & Drink Shop
3,M5P,Central Toronto,Forest Hill North,43.696948,-79.411307,3,Sushi Restaurant,Trail,Jewelry Store,Park,Food & Drink Shop,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Yoga Studio
4,M5P,Central Toronto,Forest Hill West,43.696948,-79.411307,3,Sushi Restaurant,Trail,Jewelry Store,Park,Food & Drink Shop,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Yoga Studio


### Visualize the resulting clusters.

In [122]:
# Visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(central_toronto_merged['Latitude'], central_toronto_merged['Longitude'], central_toronto_merged['Neighborhood'], central_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

### Five clusters are below.

In [123]:
central_toronto_merged.loc[central_toronto_merged['Cluster Labels'] == 0, central_toronto_merged.columns[[1] + list(range(5, central_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Toronto,0,Sandwich Place,Asian Restaurant,Gym,Breakfast Spot,Dance Studio,Burger Joint,Hotel,Park,Pizza Place,Food & Drink Shop
5,Central Toronto,0,Coffee Shop,Sporting Goods Shop,Sandwich Place,Gift Shop,Fast Food Restaurant,Diner,Dessert Shop,Metro Station,Mexican Restaurant,Park
6,Central Toronto,0,Sandwich Place,Café,Coffee Shop,Pizza Place,American Restaurant,History Museum,Indian Restaurant,Jewish Restaurant,Liquor Store,Park
7,Central Toronto,0,Sandwich Place,Café,Coffee Shop,Pizza Place,American Restaurant,History Museum,Indian Restaurant,Jewish Restaurant,Liquor Store,Park
8,Central Toronto,0,Sandwich Place,Café,Coffee Shop,Pizza Place,American Restaurant,History Museum,Indian Restaurant,Jewish Restaurant,Liquor Store,Park
9,Central Toronto,0,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Restaurant,Café,Coffee Shop,Sushi Restaurant,Park,Dance Studio
12,Central Toronto,0,Pub,Coffee Shop,American Restaurant,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Medical Center,Pizza Place,Convenience Store
13,Central Toronto,0,Pub,Coffee Shop,American Restaurant,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Medical Center,Pizza Place,Convenience Store
14,Central Toronto,0,Pub,Coffee Shop,American Restaurant,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Medical Center,Pizza Place,Convenience Store
15,Central Toronto,0,Pub,Coffee Shop,American Restaurant,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Medical Center,Pizza Place,Convenience Store


In [124]:
central_toronto_merged.loc[central_toronto_merged['Cluster Labels'] == 1, central_toronto_merged.columns[[1] + list(range(5, central_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,1,Garden,Yoga Studio,Dance Studio,History Museum,Gym,Greek Restaurant,Gourmet Shop,Gift Shop,Fried Chicken Joint,Food & Drink Shop


In [125]:
central_toronto_merged.loc[central_toronto_merged['Cluster Labels'] == 2, central_toronto_merged.columns[[1] + list(range(5, central_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Central Toronto,2,Restaurant,Gym,Playground,Tennis Court,Flower Shop,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fried Chicken Joint
11,Central Toronto,2,Restaurant,Gym,Playground,Tennis Court,Flower Shop,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fried Chicken Joint


In [126]:
central_toronto_merged.loc[central_toronto_merged['Cluster Labels'] == 3, central_toronto_merged.columns[[1] + list(range(5, central_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Central Toronto,3,Sushi Restaurant,Trail,Jewelry Store,Park,Food & Drink Shop,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Yoga Studio
4,Central Toronto,3,Sushi Restaurant,Trail,Jewelry Store,Park,Food & Drink Shop,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Yoga Studio


In [127]:
central_toronto_merged.loc[central_toronto_merged['Cluster Labels'] == 4, central_toronto_merged.columns[[1] + list(range(5, central_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,4,Bus Line,Park,Swim School,Yoga Studio,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint
